# Getting New Data

In [ ]:
#Josh's code
basegame = 4108071572
url = 'https://americas.api.riotgames.com/'
name = 'NoNoYogi'
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
puuid =response.json()['puuid']
pulls = 0
count = 1
while pulls != 100:
    
    query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={count}&count=1&api_key={api_key}'
    response = requests.get(url+query)
    
    for game in response.json():
        if int(game[4:]) > basegame:
            print("higher")
        else:
            print('lower')
    count +=1
    pulls +=1

In [ ]:
#Jared's code
def get_new_match_ids(puuids):
    url = 'https://americas.api.riotgames.com/'
    for puuid in puuids:
        i = 1
        while i < 20:
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i-1}&count={i}&api_key={api_key}'
            response = requests.get(url+query)
            match = response.json()
            match_num = int(match[0].strip('NA1_'))
            if match_num > 4108071572:
                i += 1
                match_list.append(match)
            else:
                break

### Combine the Code Above Into One Function

In [80]:
def get_new_patch_data(name_list, api_key):
    """
    This function takes in a list of usernames that are in the same tier and iterates 
    through them to find their puuids.
    It will then use those puuids to grab all of their match ids that are in the newest patch.
    From there, the function will iterate through all of the match ids and grab the timeline
    data for each match as well as the other game data. 
    Finally, this function will incrementally save the returned json information every 10 times.
    This prevents the loss of all data if the function errors out at some point.
    This function returns nothing.
    """
    
    #Set up the basegame number. Any match_ids below this number are part of the old patch
    #and will not be included. We will use the basegame var for comparisons in the loop
    basegame = 4107000001
    
    #Set up the base url for grabbing match_ids
    match_url = 'https://americas.api.riotgames.com/'
    
    #Set up the base url for grabbing puuids
    puuid_base_url = 'https://na1.api.riotgames.com/'
    
    #Set up another counter to keep track of how many users have been completed
    n = 0
    
    #Set up a counter to keep track of how many json files have been completed in total
    num_json = 0
    
    #Set up one more counter to keep track of how many pulls we have made.
    #Can't go over 100 pulls every 2 minutes.
    num_pulls = 0
    
    #Set up empty lists to store timeline and other game data
    timeline_data = []
    game_data = []
    
    #Loop through each name in the given name_list
    #First grab the puuid, then grab the match_ids, then actual match json info
    for name in name_list:
        
        #for testing
        print(f'Beginning Loop for user {name}')
        
        #Update n var
        n += 1
        
        #Leave a status update after every 100 users
        if n % 100 == 0:
            print(f'Completed {n} users of {len(name_list)}!\n')
        
        ########################## Get puuids ##########################
        
        #Set up the query
        query = f'lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
        
        #Get the response
        response = requests.get(puuid_base_url + query)
        
        #Update pull count and sleep for 150 seconds if necessary
        num_pulls += 1
        
        if num_pulls % 100 == 0:
            print('\nHit 100 pulls! Sleeping for 150 seconds.')
            sleep(150)
            print('Continuing!\n')
        
        #Check the response code and leave message if something goes wrong
        if response.status_code != 200:
            print(f'Something went wrong! Status Code: {response.status_code}. Did not get puuid for user: {name}')
            print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
            continue
        
        #Grab the user's puuid
        puuid = response.json()['puuid']
        
        #For testing
        print(f'\t-Got the puuid for user {name}.')
        
        ########################## Get Match IDs #########################
        
        #Now that we have a single puuid, grab all the match ids for that user that are greater
        #than the basegame value. These match_ids will be for the newest patch.
        
        #Set up a counter
        i = 0
        
        #This loop will be set up to run indefinitely. Be sure to break out of it when 
        #Done with the current user
        while i >= 0:
            #Set up the query to get a match_id
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i}&count=1&api_key={api_key}'
            
            #Get the response
            response = requests.get(match_url + query)
            
            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response status code and leave message if something went wrong
            if response.status_code != 200:
                print(f'Something went wrong! Status Code: {response.status_code}. Did not get Match_ID for user: {name}')
                print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
                break
                
            #Grab the match_id
            match = response.json()
            
            #Verify there's something in it
            if len(match) == 0:
                print(f'\t-No match ID for user {name}. Skipping this user!')
                break
            
            #Convert match_id to integer
            match_num = int(match[0].strip('NA1_'))
            
            #Compare current match_id to basegame var.
            #Skip if match_id is less than basegame
            if match_num < basegame:
                print(f'\t-Completed getting all current patch Match IDs for user {name}')
                print(f'\t-There are {len(name_list) - n} users left.')
                break

            ####################### Get Match Info ##########################
            
            #Now that we know the current match_id is part of the newest patch,
            #We can retrieve the actual match info. Keep in mind this is still part
            #of the while loop above
                
            #Every 250 entries, save the lists as json files and reset the lists
            if len(timeline_data) % 250 == 0:    
                #Save timeline data.
                #First, convert it to a df
                timeline_df = pd.DataFrame(timeline_data)

                #Save as json file. Use username string to identify whose file it is
                timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created timeline_data_user_{name}_end_{num_json}.json')

                #Save other game data
                #First, convert it to a df
                game_df = pd.DataFrame(game_data)

                #Save as json file. Use username string to identify whose file it is
                game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created other_game_data_user_{name}_end_{num_json}.json\n')
                
                #Now reset the temp lists
                timeline_data = []
                game_data = []
            
        
            ############################# TIMELINE DATA #######################
            #Set up timeline url
            timeline_query = f'lol/match/v5/matches/{match[0]}/timeline/?api_key={api_key}'

            #Grab timeline json data
            timeline_response = requests.get(match_url + timeline_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message
            if timeline_response.status_code != 200:
                print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID.\n')
                continue

            #Turn it into json format
            timeline_json = timeline_response.json()

            #Append this data to the timeline_data list
            timeline_data.append(timeline_json)
            
            ############################# OTHER GAME DATA #######################
            #Set up game data url
            game_query = f'lol/match/v5/matches/{match[0]}?api_key={api_key}'

            #Grab game json data
            game_response = requests.get(match_url + game_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message.
            if game_response.status_code != 200:
                print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')

                #Remove the last entry in the timeline_data list
                timeline_data.pop()
                continue

            #Turn it into json format
            game_json = game_response.json()

            #Append this data to the game_data list
            game_data.append(game_json)
            
            #Update num_json
            num_json += 1
            
            #Update i
            i += 1
            
        #Tell the user how many match_ids were used for the current user
        print(f'\t-Completed {i} Match IDs for user {name}\n')
    
    #Save any json data that was not saved at the end of the for loop
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)

    #Save as json file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created timeline_data_user_{name}_end_{num_json}.json')

    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)

    #Save as json file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created other_game_data_user_{name}_end_{num_json}.json')
    
    #Complete! Leave a message
    print('All users completed! Follow suit.')
            

In [1]:
#For testing, import the user name
from pro_names import pro_names
from env import api_key
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key
from time import sleep

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [54]:
test_list = pro_names[:10]
test_list

['Huni',
 'CoreJJ',
 'Impact',
 'Contractz',
 'Jensen',
 'Sneaky',
 'Smoothie',
 'Westrice',
 'Darshan',
 'Dardoch']

In [55]:
last_timeline_data, last_game_data = get_new_patch_data(test_list, api_key)

Beginning Loop for user Huni
	-Got the puuid for user Huni.
	-Completed getting all current patch Match IDs for user Huni
	-There are 9 users left.
	-Completed 0 Match IDs for user Huni

Beginning Loop for user CoreJJ
	-Got the puuid for user CoreJJ.
	-Completed getting all current patch Match IDs for user CoreJJ
	-There are 8 users left.
	-Completed 0 Match IDs for user CoreJJ

Beginning Loop for user Impact
	-Got the puuid for user Impact.
	-Completed getting all current patch Match IDs for user Impact
	-There are 7 users left.
	-Completed 0 Match IDs for user Impact

Beginning Loop for user Contractz
	-Got the puuid for user Contractz.
	-Completed getting all current patch Match IDs for user Contractz
	-There are 6 users left.
	-Completed 0 Match IDs for user Contractz

Beginning Loop for user Jensen
	-Got the puuid for user Jensen.
	-Created timeline_data_user_Jensen_end_0.json
	-Created other_game_data_user_Jensen_end_0.json

	-Created timeline_data_user_Jensen_end_10.json
	-Creat

In [56]:
len(last_timeline_data)

3

In [57]:
timeline_data = pd.read_json('timeline_data_user_Jensen_end_10.json')
game_data = pd.read_json('other_game_data_user_Jensen_end_10.json')

In [58]:
#Convert to list of dicts
timeline_data = timeline_data.to_dict(orient = 'records')
game_data = game_data.to_dict(orient = 'records')

In [59]:
import prepare

In [60]:
df = prepare.prepare(timeline_data, game_data, 20)

Finished with: 0 of 10
Finished with: 1 of 10
Finished with: 2 of 10
Finished with: 3 of 10
Skipping: 4 due to <20 min or not classic
Finished with: 5 of 10
Finished with: 6 of 10
Skipping: 7 due to <20 min or not classic
Finished with: 8 of 10
Finished with: 9 of 10


In [64]:
df = prepare.prep(df)

In [66]:
df.shape

(8, 207)

In [68]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 207 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    killsplayer_1                          float64
 1    killsplayer_2                          float64
 2    killsplayer_4                          float64
 3    killsplayer_5                          float64
 4    killsplayer_6                          float64
 5    killsplayer_7                          float64
 6    killsplayer_8                          float64
 7    killsplayer_9                          float64
 8    killsplayer_10                         float64
 9    deathsplayer_1                         float64
 10   deathsplayer_2                         float64
 11   deathsplayer_3                         float64
 12   deathsplayer_4                         float64
 13   deathsplayer_5                         float64
 14   deathsplayer_6                         float

# Get the New Data

Now that the function has been completed, get the desired data. Joshua C. is gathering the data for the pro players. I will get the data using the names from Jared's list.

In [77]:
#Load the names csv
name_list = pd.read_csv('2640_players.csv', index_col = [0])
name_list

,players
0,Ahwii
1,Blazze
2,Prabhably
3,Dark Rose Vayne
4,hyperton
...,...
2636,Denathor
2637,Impmon
2638,personnel office
2639,EBIC GAMER


__How Many?__

Since there are over 2600 names, I will only attempt to gather for the first 1000 of them. I will save a json file after every 250 json entries.

In [78]:
name_list = list(name_list.players)

In [82]:
name_list = name_list[:1000]

In [84]:
#Run the function
get_new_patch_data(name_list, api_key)

Beginning Loop for user Ahwii
	-Got the puuid for user Ahwii.
	-Created timeline_data_user_Ahwii_end_0.json
	-Created other_game_data_user_Ahwii_end_0.json

	-Completed getting all current patch Match IDs for user Ahwii
	-There are 999 users left.
	-Completed 11 Match IDs for user Ahwii

Beginning Loop for user Blazze
	-Got the puuid for user Blazze.
	-Completed getting all current patch Match IDs for user Blazze
	-There are 998 users left.
	-Completed 2 Match IDs for user Blazze

Beginning Loop for user Prabhably
	-Got the puuid for user Prabhably.
	-Completed getting all current patch Match IDs for user Prabhably
	-There are 997 users left.
	-Completed 0 Match IDs for user Prabhably

Beginning Loop for user Dark Rose Vayne
	-Got the puuid for user Dark Rose Vayne.
	-Completed getting all current patch Match IDs for user Dark Rose Vayne
	-There are 996 users left.
	-Completed 0 Match IDs for user Dark Rose Vayne

Beginning Loop for user hyperton
	-Got the puuid for user hyperton.
	-Co

	-Completed getting all current patch Match IDs for user Shieda Kayn
	-There are 964 users left.
	-Completed 2 Match IDs for user Shieda Kayn

Beginning Loop for user HelImet
	-Got the puuid for user HelImet.
	-Completed getting all current patch Match IDs for user HelImet
	-There are 963 users left.
	-Completed 2 Match IDs for user HelImet

Beginning Loop for user Shroomologist
	-Got the puuid for user Shroomologist.
	-Completed getting all current patch Match IDs for user Shroomologist
	-There are 962 users left.
	-Completed 4 Match IDs for user Shroomologist

Beginning Loop for user align my chakras
	-Got the puuid for user align my chakras.
	-Completed getting all current patch Match IDs for user align my chakras
	-There are 961 users left.
	-Completed 0 Match IDs for user align my chakras

Beginning Loop for user Injustable
	-Got the puuid for user Injustable.
	-Completed getting all current patch Match IDs for user Injustable
	-There are 960 users left.
	-Completed 0 Match IDs fo

	-Got the puuid for user Han Yi.
	-Completed getting all current patch Match IDs for user Han Yi
	-There are 925 users left.
	-Completed 0 Match IDs for user Han Yi

Beginning Loop for user LB gotoe11
	-Got the puuid for user LB gotoe11.
	-Completed getting all current patch Match IDs for user LB gotoe11
	-There are 924 users left.
	-Completed 2 Match IDs for user LB gotoe11

Beginning Loop for user Last Savant
	-Got the puuid for user Last Savant.
	-Completed getting all current patch Match IDs for user Last Savant
	-There are 923 users left.
	-Completed 0 Match IDs for user Last Savant

Beginning Loop for user Windperson
	-Got the puuid for user Windperson.
	-Completed getting all current patch Match IDs for user Windperson
	-There are 922 users left.
	-Completed 2 Match IDs for user Windperson

Beginning Loop for user Chime2
	-Got the puuid for user Chime2.
	-Completed getting all current patch Match IDs for user Chime2
	-There are 921 users left.
	-Completed 0 Match IDs for user Ch

	-Got the puuid for user Sum.
	-Completed getting all current patch Match IDs for user Sum
	-There are 887 users left.
	-Completed 1 Match IDs for user Sum

Beginning Loop for user Utopian Mindset
	-Got the puuid for user Utopian Mindset.
	-Completed getting all current patch Match IDs for user Utopian Mindset
	-There are 886 users left.
	-Completed 0 Match IDs for user Utopian Mindset

Beginning Loop for user Aireech
	-Got the puuid for user Aireech.
	-Completed getting all current patch Match IDs for user Aireech
	-There are 885 users left.
	-Completed 1 Match IDs for user Aireech

Beginning Loop for user GDOG
	-Got the puuid for user GDOG.
	-Completed getting all current patch Match IDs for user GDOG
	-There are 884 users left.
	-Completed 0 Match IDs for user GDOG

Beginning Loop for user jchan3580
	-Got the puuid for user jchan3580.
	-Completed getting all current patch Match IDs for user jchan3580
	-There are 883 users left.
	-Completed 9 Match IDs for user jchan3580

Beginning L

	-Got the puuid for user CY Blood.
	-Completed getting all current patch Match IDs for user CY Blood
	-There are 849 users left.
	-Completed 13 Match IDs for user CY Blood

Beginning Loop for user Vakin
	-Got the puuid for user Vakin.
	-Completed getting all current patch Match IDs for user Vakin
	-There are 848 users left.
	-Completed 3 Match IDs for user Vakin

Beginning Loop for user coblsobr
	-Got the puuid for user coblsobr.
	-Completed getting all current patch Match IDs for user coblsobr
	-There are 847 users left.
	-Completed 6 Match IDs for user coblsobr

Beginning Loop for user C9 Avril

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Got the puuid for user C9 Avril.
	-Completed getting all current patch Match IDs for user C9 Avril
	-There are 846 users left.
	-Completed 2 Match IDs for user C9 Avril

Beginning Loop for user Beginning
	-Got the puuid for user Beginning.
	-Completed getting all current patch Match IDs for user Beginning
	-There are 845 users left.
	-Co

...

	-Got the puuid for user Chuty.
	-Completed getting all current patch Match IDs for user Chuty
	-There are 773 users left.
	-Completed 2 Match IDs for user Chuty

Beginning Loop for user Azog
	-Got the puuid for user Azog.
	-Completed getting all current patch Match IDs for user Azog
	-There are 772 users left.
	-Completed 2 Match IDs for user Azog

Beginning Loop for user Abedz

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Got the puuid for user Abedz.
	-Completed getting all current patch Match IDs for user Abedz
	-There are 771 users left.
	-Completed 1 Match IDs for user Abedz

Beginning Loop for user JackySan
	-Got the puuid for user JackySan.
	-Completed getting all current patch Match IDs for user JackySan
	-There are 770 users left.
	-Completed 5 Match IDs for user JackySan

Beginning Loop for user Elle Me Dit
	-Got the puuid for user Elle Me Dit.
	-Completed getting all current patch Match IDs for user Elle Me Dit
	-There are 769 users left.
	-Completed 0 Match IDs 

	-Got the puuid for user xHAYDEN.
	-Completed getting all current patch Match IDs for user xHAYDEN
	-There are 735 users left.
	-Completed 0 Match IDs for user xHAYDEN

Beginning Loop for user 04 Jug
	-Got the puuid for user 04 Jug.
	-Completed getting all current patch Match IDs for user 04 Jug
	-There are 734 users left.
	-Completed 2 Match IDs for user 04 Jug

Beginning Loop for user TEMP 27236133
	-Got the puuid for user TEMP 27236133.
	-Completed getting all current patch Match IDs for user TEMP 27236133
	-There are 733 users left.
	-Completed 0 Match IDs for user TEMP 27236133

Beginning Loop for user Dawiid
	-Got the puuid for user Dawiid.
	-Completed getting all current patch Match IDs for user Dawiid
	-There are 732 users left.
	-Completed 0 Match IDs for user Dawiid

Beginning Loop for user wzy
	-Got the puuid for user wzy.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user wzy
	-There are 731 users left.
	-Complete

	-Got the puuid for user Ring Men.
	-Completed getting all current patch Match IDs for user Ring Men
	-There are 697 users left.
	-Completed 2 Match IDs for user Ring Men

Beginning Loop for user Cpt Goofy
	-Got the puuid for user Cpt Goofy.
	-Completed getting all current patch Match IDs for user Cpt Goofy
	-There are 696 users left.
	-Completed 0 Match IDs for user Cpt Goofy

Beginning Loop for user Devasis
	-Got the puuid for user Devasis.
	-Completed getting all current patch Match IDs for user Devasis
	-There are 695 users left.
	-Completed 16 Match IDs for user Devasis

Beginning Loop for user Your Ruined King
	-Got the puuid for user Your Ruined King.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user Your Ruined King
	-There are 694 users left.
	-Completed 2 Match IDs for user Your Ruined King

Beginning Loop for user under ur spell
	-Got the puuid for user under ur spell.
	-Completed getting all current patch Match I

	-Got the puuid for user Assisted.
	-Completed getting all current patch Match IDs for user Assisted
	-There are 659 users left.
	-Completed 1 Match IDs for user Assisted

Beginning Loop for user Blueyoshi6
	-Got the puuid for user Blueyoshi6.
	-Completed getting all current patch Match IDs for user Blueyoshi6
	-There are 658 users left.
	-Completed 1 Match IDs for user Blueyoshi6

Beginning Loop for user Rhaastafarian
	-Got the puuid for user Rhaastafarian.
	-Completed getting all current patch Match IDs for user Rhaastafarian
	-There are 657 users left.
	-Completed 6 Match IDs for user Rhaastafarian

Beginning Loop for user Trieuly
	-Got the puuid for user Trieuly.
	-Completed getting all current patch Match IDs for user Trieuly
	-There are 656 users left.
	-Completed 0 Match IDs for user Trieuly

Beginning Loop for user DanDanQAQ
	-Got the puuid for user DanDanQAQ.
	-Completed getting all current patch Match IDs for user DanDanQAQ
	-There are 655 users left.
	-Completed 3 Match IDs 

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [86]:
#Completed 320 names in name_list before it errored out.
#Create new list starting at index 321 and restart the function
name_list = name_list[321:1000]

In [88]:
#Run the function
get_new_patch_data(name_list, api_key)

Beginning Loop for user Optimistic Ego
	-Got the puuid for user Optimistic Ego.
	-Created timeline_data_user_Optimistic Ego_end_0.json
	-Created other_game_data_user_Optimistic Ego_end_0.json


Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user Optimistic Ego
	-There are 678 users left.
	-Completed 40 Match IDs for user Optimistic Ego

Beginning Loop for user step1v9
	-Got the puuid for user step1v9.
	-Completed getting all current patch Match IDs for user step1v9
	-There are 677 users left.
	-Completed 0 Match IDs for user step1v9

Beginning Loop for user EvilWeave
	-Got the puuid for user EvilWeave.
	-Completed getting all current patch Match IDs for user EvilWeave
	-There are 676 users left.
	-Completed 6 Match IDs for user EvilWeave

Beginning Loop for user SpazzJG
	-Got the puuid for user SpazzJG.
	-Completed getting all current patch Match IDs for user SpazzJG
	-There are 675 users left.
	-Completed 0 Match IDs for user 

	-Got the puuid for user love must remain.
	-Completed getting all current patch Match IDs for user love must remain
	-There are 640 users left.
	-Completed 0 Match IDs for user love must remain

Beginning Loop for user Mr Comet
	-Got the puuid for user Mr Comet.
	-Completed getting all current patch Match IDs for user Mr Comet
	-There are 639 users left.
	-Completed 11 Match IDs for user Mr Comet

Beginning Loop for user Abused Mid
	-Got the puuid for user Abused Mid.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user Abused Mid
	-There are 638 users left.
	-Completed 9 Match IDs for user Abused Mid

Beginning Loop for user PlsBuffNeeko
	-Got the puuid for user PlsBuffNeeko.
	-Completed getting all current patch Match IDs for user PlsBuffNeeko
	-There are 637 users left.
	-Completed 7 Match IDs for user PlsBuffNeeko

Beginning Loop for user Calm Antebellum
	-Got the puuid for user Calm Antebellum.
	-Completed getting all cur

	-Got the puuid for user Titan Dweevil.
	-Completed getting all current patch Match IDs for user Titan Dweevil
	-There are 602 users left.
	-Completed 2 Match IDs for user Titan Dweevil

Beginning Loop for user Shiba lnu
	-Got the puuid for user Shiba lnu.
	-Completed getting all current patch Match IDs for user Shiba lnu
	-There are 601 users left.
	-Completed 0 Match IDs for user Shiba lnu

Beginning Loop for user King Veer
	-Got the puuid for user King Veer.
	-Completed getting all current patch Match IDs for user King Veer
	-There are 600 users left.
	-Completed 6 Match IDs for user King Veer

Beginning Loop for user Skarmori
	-Got the puuid for user Skarmori.
	-Completed getting all current patch Match IDs for user Skarmori
	-There are 599 users left.
	-Completed 0 Match IDs for user Skarmori

Beginning Loop for user Rocktank10
	-Got the puuid for user Rocktank10.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user Rockta

	-Got the puuid for user JDPayne.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user JDPayne
	-There are 563 users left.
	-Completed 7 Match IDs for user JDPayne

Beginning Loop for user Unleashed1
	-Got the puuid for user Unleashed1.
	-Completed getting all current patch Match IDs for user Unleashed1
	-There are 562 users left.
	-Completed 0 Match IDs for user Unleashed1

Beginning Loop for user Fiora
	-Got the puuid for user Fiora.
	-Completed getting all current patch Match IDs for user Fiora
	-There are 561 users left.
	-Completed 0 Match IDs for user Fiora

Beginning Loop for user Shoubee
	-Got the puuid for user Shoubee.
	-Completed getting all current patch Match IDs for user Shoubee
	-There are 560 users left.
	-Completed 0 Match IDs for user Shoubee

Beginning Loop for user Nim Chimpsky
	-Got the puuid for user Nim Chimpsky.
	-Completed getting all current patch Match IDs for user Nim Chimpsky
	-There are 559 users l

	-Got the puuid for user MudBone69.
	-Completed getting all current patch Match IDs for user MudBone69
	-There are 524 users left.
	-Completed 2 Match IDs for user MudBone69

Beginning Loop for user loljanan
	-Got the puuid for user loljanan.
	-Completed getting all current patch Match IDs for user loljanan
	-There are 523 users left.
	-Completed 0 Match IDs for user loljanan

Beginning Loop for user KingClueless
	-Got the puuid for user KingClueless.
	-Completed getting all current patch Match IDs for user KingClueless
	-There are 522 users left.
	-Completed 4 Match IDs for user KingClueless

Beginning Loop for user Cpt Underpants
	-Got the puuid for user Cpt Underpants.
	-Completed getting all current patch Match IDs for user Cpt Underpants
	-There are 521 users left.
	-Completed 4 Match IDs for user Cpt Underpants

Beginning Loop for user Alfaren
	-Got the puuid for user Alfaren.
	-Completed getting all current patch Match IDs for user Alfaren
	-There are 520 users left.
	-Completed

...


Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Got the puuid for user Goth Squirrel.
	-Completed getting all current patch Match IDs for user Goth Squirrel
	-There are 184 users left.
	-Completed 8 Match IDs for user Goth Squirrel

Beginning Loop for user egirl ryze 
	-Got the puuid for user egirl ryze .
	-Completed getting all current patch Match IDs for user egirl ryze 
	-There are 183 users left.
	-Completed 1 Match IDs for user egirl ryze 

Beginning Loop for user year of lourlo
	-Got the puuid for user year of lourlo.
	-Completed getting all current patch Match IDs for user year of lourlo
	-There are 182 users left.
	-Completed 14 Match IDs for user year of lourlo

Beginning Loop for user C9 Copy
	-Got the puuid for user C9 Copy.
	-Completed getting all current patch Match IDs for user C9 Copy
	-There are 181 users left.
	-Completed 0 Match IDs for user C9 Copy

Beginning Loop for user iybc9o2 q
	-Got the puuid for user iybc9o2 q.
	-Completed getting all current patch Ma

	-Got the puuid for user Zahnet.
	-Completed getting all current patch Match IDs for user Zahnet
	-There are 147 users left.
	-Completed 3 Match IDs for user Zahnet

Beginning Loop for user F0D
	-Got the puuid for user F0D.
	-Completed getting all current patch Match IDs for user F0D
	-There are 146 users left.
	-Completed 1 Match IDs for user F0D

Beginning Loop for user garlic naan34
	-Got the puuid for user garlic naan34.
	-Completed getting all current patch Match IDs for user garlic naan34
	-There are 145 users left.
	-Completed 0 Match IDs for user garlic naan34

Beginning Loop for user Scenari0
	-Got the puuid for user Scenari0.
	-Completed getting all current patch Match IDs for user Scenari0
	-There are 144 users left.
	-Completed 0 Match IDs for user Scenari0

Beginning Loop for user Kimrin
	-Got the puuid for user Kimrin.
	-Completed getting all current patch Match IDs for user Kimrin
	-There are 143 users left.
	-Completed 0 Match IDs for user Kimrin

Beginning Loop for use

	-Got the puuid for user Super Big Ego.
	-Completed getting all current patch Match IDs for user Super Big Ego
	-There are 109 users left.
	-Completed 4 Match IDs for user Super Big Ego

Beginning Loop for user Someonehastodoit
	-Got the puuid for user Someonehastodoit.
	-Created timeline_data_user_Someonehastodoit_end_2000.json
	-Created other_game_data_user_Someonehastodoit_end_2000.json

	-Completed getting all current patch Match IDs for user Someonehastodoit
	-There are 108 users left.
	-Completed 1 Match IDs for user Someonehastodoit

Beginning Loop for user dumpa
	-Got the puuid for user dumpa.
	-Completed getting all current patch Match IDs for user dumpa
	-There are 107 users left.
	-Completed 5 Match IDs for user dumpa

Beginning Loop for user sahori
	-Got the puuid for user sahori.
	-Completed getting all current patch Match IDs for user sahori
	-There are 106 users left.
	-Completed 0 Match IDs for user sahori

Beginning Loop for user sweetvictoree
	-Got the puuid for user 

	-Got the puuid for user 2oon.
	-Completed getting all current patch Match IDs for user 2oon
	-There are 72 users left.
	-Completed 6 Match IDs for user 2oon

Beginning Loop for user Vultix
	-Got the puuid for user Vultix.
	-Completed getting all current patch Match IDs for user Vultix
	-There are 71 users left.
	-Completed 4 Match IDs for user Vultix

Beginning Loop for user CookiesforSanta
	-Got the puuid for user CookiesforSanta.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user CookiesforSanta
	-There are 70 users left.
	-Completed 22 Match IDs for user CookiesforSanta

Beginning Loop for user 02 16 2017
	-Got the puuid for user 02 16 2017.
	-Completed getting all current patch Match IDs for user 02 16 2017
	-There are 69 users left.
	-Completed 4 Match IDs for user 02 16 2017

Beginning Loop for user Bellow1
	-Got the puuid for user Bellow1.
	-Completed getting all current patch Match IDs for user Bellow1
	-There are 68

	-Completed getting all current patch Match IDs for user dfgtredfg4
	-There are 35 users left.
	-Completed 0 Match IDs for user dfgtredfg4

Beginning Loop for user Shrimp Cpt
	-Got the puuid for user Shrimp Cpt.
	-Completed getting all current patch Match IDs for user Shrimp Cpt
	-There are 34 users left.
	-Completed 0 Match IDs for user Shrimp Cpt

Beginning Loop for user Nunu Main
	-Got the puuid for user Nunu Main.
	-Completed getting all current patch Match IDs for user Nunu Main
	-There are 33 users left.
	-Completed 10 Match IDs for user Nunu Main

Beginning Loop for user BratEvolved
	-Got the puuid for user BratEvolved.
	-Completed getting all current patch Match IDs for user BratEvolved
	-There are 32 users left.
	-Completed 0 Match IDs for user BratEvolved

Beginning Loop for user ADC IS GAPPED
	-Got the puuid for user ADC IS GAPPED.
	-Completed getting all current patch Match IDs for user ADC IS GAPPED
	-There are 31 users left.
	-Completed 3 Match IDs for user ADC IS GAPPED


# Join All of the New Json Files Together

In [2]:
#Import os and try to programatically grab the json file names
import os

In [2]:
timeline_files = [ x for x in os.listdir('./new_data') if x.startswith("timeline") ]
timeline_files.sort()
timeline_files

['timeline_data_user_Bruder_end_250.json',
 'timeline_data_user_Derek HH_end_750.json',
 'timeline_data_user_Eighty Six_end_1000.json',
 'timeline_data_user_Kingicepop_end_1250.json',
 'timeline_data_user_LEO99_end_1750.json',
 'timeline_data_user_Okuri_end_2397.json',
 'timeline_data_user_Optimistic Ego_end_0.json',
 'timeline_data_user_REN JIAN SHI GE_end_750.json',
 'timeline_data_user_Someonehastodoit_end_2000.json',
 'timeline_data_user_Sopachi_end_500.json',
 'timeline_data_user_TL Yeon_end_1500.json',
 'timeline_data_user_Zayno_end_1000.json',
 'timeline_data_user_baekho_end_2250.json',
 'timeline_data_user_scooterguy69_end_500.json',
 'timeline_data_user_wx NAtongtiandai_end_250.json']

In [3]:
other_data_files = [ x for x in os.listdir('./new_data') if x.startswith("other") ]
other_data_files.sort()
other_data_files

['other_game_data_user_Bruder_end_250.json',
 'other_game_data_user_Derek HH_end_750.json',
 'other_game_data_user_Eighty Six_end_1000.json',
 'other_game_data_user_Kingicepop_end_1250.json',
 'other_game_data_user_LEO99_end_1750.json',
 'other_game_data_user_Okuri_end_2397.json',
 'other_game_data_user_Optimistic Ego_end_0.json',
 'other_game_data_user_REN JIAN SHI GE_end_750.json',
 'other_game_data_user_Someonehastodoit_end_2000.json',
 'other_game_data_user_Sopachi_end_500.json',
 'other_game_data_user_TL Yeon_end_1500.json',
 'other_game_data_user_Zayno_end_1000.json',
 'other_game_data_user_baekho_end_2250.json',
 'other_game_data_user_scooterguy69_end_500.json',
 'other_game_data_user_wx NAtongtiandai_end_250.json']

In [1]:
import pandas as pd
import acquire
import prepare

In [10]:
#Create a function to loop through these files and prepare them
#Will need an empty df to store the final df
complete_df = pd.DataFrame()

for i in range(0, len(timeline_files[:2])):
    #Need to read both of them into dataframes and then convert to list of dicts
    timeline_list = pd.read_json('./new_data/' + timeline_files[i])
    game_list = pd.read_json('./new_data/' + other_data_files[i])

    #Convert to list of dicts
    timeline_list = timeline_list.to_dict(orient = 'records')
    game_list = game_list.to_dict(orient = 'records')

    #Now feed them into Joshua C's prepare file. Extract data at the 15 minute mark
    temp_df = acquire.extract(timeline_list, game_list, 15)
    print(f'Extracted the data! Entries: {len(temp_df)}')

    #Now prepare the data
    temp_df = prepare.prepare(temp_df)
    print(f'Prepared the data! Entries: {len(temp_df)}')

    #Now append the temp_df to the complete_df
    complete_df = complete_df.append(temp_df, ignore_index = True)
    print(f'Appended the data! Complete DataFrame Entries: {len(complete_df)}')

#Drop duplicates and return the prepared df
complete_df = complete_df.drop_duplicates()

Finished with: 0 of 249
Finished with: 1 of 249
Finished with: 2 of 249
Finished with: 3 of 249
Skipping: 4 due to <20 min or not classic
Skipping: 5 due to <20 min or not classic
Skipping: 6 due to <20 min or not classic
Skipping: 7 due to <20 min or not classic
Finished with: 8 of 249
Finished with: 9 of 249
Skipping: 10 due to <20 min or not classic
Finished with: 11 of 249
Finished with: 12 of 249
Finished with: 13 of 249
Finished with: 14 of 249
Skipping: 15 due to <20 min or not classic
Finished with: 16 of 249
Finished with: 17 of 249
Finished with: 18 of 249
Skipping: 19 due to <20 min or not classic
Finished with: 20 of 249
Finished with: 21 of 249
Skipping: 22 due to <20 min or not classic
Finished with: 23 of 249
Finished with: 24 of 249
Skipping: 25 due to <20 min or not classic
Finished with: 26 of 249
Finished with: 27 of 249
Finished with: 28 of 249
Finished with: 29 of 249
Finished with: 30 of 249
Skipping: 31 due to <20 min or not classic
Finished with: 32 of 249
Finis

Finished with: 242 of 249
Finished with: 243 of 249
Skipping: 244 due to <20 min or not classic
Finished with: 245 of 249
Skipping: 246 due to <20 min or not classic
Finished with: 247 of 249
Skipping: 248 due to <20 min or not classic
Finished with: 249 of 249
Extracted the data! Entries: 127
Prepared the data! Entries: 2
Appended the data! Complete DataFrame Entries: 127
Finished with: 0 of 249
Skipping: 1 due to <20 min or not classic
Skipping: 2 due to <20 min or not classic
Finished with: 3 of 249
Finished with: 4 of 249
Finished with: 5 of 249
Finished with: 6 of 249
Finished with: 7 of 249
Skipping: 8 due to <20 min or not classic
Skipping: 9 due to <20 min or not classic
Finished with: 10 of 249
Finished with: 11 of 249
Skipping: 12 due to <20 min or not classic
Finished with: 13 of 249
Finished with: 14 of 249
Finished with: 15 of 249
Skipping: 16 due to <20 min or not classic
Skipping: 17 due to <20 min or not classic
Skipping: 18 due to <20 min or not classic
Finished with: 

Finished with: 232 of 249
Finished with: 233 of 249
Finished with: 234 of 249
Skipping: 235 due to <20 min or not classic
Finished with: 236 of 249
Skipping: 237 due to <20 min or not classic
Finished with: 238 of 249
Skipping: 239 due to <20 min or not classic
Skipping: 240 due to <20 min or not classic
Skipping: 241 due to <20 min or not classic
Finished with: 242 of 249
Finished with: 243 of 249
Finished with: 244 of 249
Skipping: 245 due to <20 min or not classic
Skipping: 246 due to <20 min or not classic
Skipping: 247 due to <20 min or not classic
Finished with: 248 of 249
Skipping: 249 due to <20 min or not classic
Extracted the data! Entries: 131
Prepared the data! Entries: 2
Appended the data! Complete DataFrame Entries: 258


In [11]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248 entries, 0 to 257
Columns: 1659 entries, killsplayer_1 to gameEndTimestamp_1637375813217
dtypes: float64(1600), int64(32), uint8(27)
memory usage: 3.1 MB


In [6]:
#Build the function
def build_extracted_df(username, path = './', time = 15):
    """
    This function will take in a path for the json files stored in your directory. 
    It will then read all of them into a list, convert them to lists of dicts, and feed
    them into the extract function. The time variable is for the extract function and determines
    at what timeframe the data will be acquired for. The username is only used for naming the
    .csv file at the end. Please use your last name to avoid confusion.
    
    This function returns a dataframe with data extracted for the specified time. 
    It also automatically saves this dataframe as a .csv.
    
    The path variable defaults to the current directory.
    The time variable defaults to the 15 minute mark.
    
    You will need to have os imported to run this function.
    """
    
    #Gather the names of the timeline json files and sort them
    #This will pull all files that start with 'timeline'
    timeline_files = [ x for x in os.listdir(path) if x.startswith("timeline") ]
    timeline_files.sort()
    
    #Gather the names of the other game data json files and sort them
    #This will pull all files that start with 'other'
    other_data_files = [ x for x in os.listdir(path) if x.startswith("other") ]
    other_data_files.sort()
    
    #Will need an empty df to store the final df
    extracted_df = pd.DataFrame()
    
    #Verify that the lists are the same length
    if len(timeline_files) != len(other_data_files):
        print('Something went wrong! The json lists are not equal!')
        print('Stopping function.')
        return

    for i in range(0, len(timeline_files)):
        #Need to read both of them into dataframes and then convert to list of dicts
        timeline_list = pd.read_json(path + timeline_files[i])
        game_list = pd.read_json(path + other_data_files[i])

        #Convert to list of dicts
        timeline_list = timeline_list.to_dict(orient = 'records')
        game_list = game_list.to_dict(orient = 'records')

        #Now feed them into Joshua C's prepare file. Extract data for the specified timeframe
        temp_df = acquire.extract(timeline_list, game_list, time)

        #Now append the temp_df to the complete_df
        extracted_df = extracted_df.append(temp_df, ignore_index = True)
            
    #Drop duplicates and return the prepared df
    extracted_df = extracted_df.drop_duplicates()
    
    #Now save the extracted_df as a .csv
    extracted_df.to_csv(f'new_extracted_data_{username}.csv', index = False)
    
    return extracted_df

In [7]:
#Test the function and build an extracted df that we can feed into the prepare function
extracted_data = build_extracted_df(username = 'smith', path = './new_data/', time = 15)

Finished with: 0 of 249
Finished with: 1 of 249
Finished with: 2 of 249
Finished with: 3 of 249
Skipping: 4 due to <20 min or not classic
Skipping: 5 due to <20 min or not classic
Skipping: 6 due to <20 min or not classic
Skipping: 7 due to <20 min or not classic
Finished with: 8 of 249
Finished with: 9 of 249
Skipping: 10 due to <20 min or not classic
Finished with: 11 of 249
Finished with: 12 of 249
Finished with: 13 of 249
Finished with: 14 of 249
Skipping: 15 due to <20 min or not classic
Finished with: 16 of 249
Finished with: 17 of 249
Finished with: 18 of 249
Skipping: 19 due to <20 min or not classic
Finished with: 20 of 249
Finished with: 21 of 249
Skipping: 22 due to <20 min or not classic
Finished with: 23 of 249
Finished with: 24 of 249
Skipping: 25 due to <20 min or not classic
Finished with: 26 of 249
Finished with: 27 of 249
Finished with: 28 of 249
Finished with: 29 of 249
Finished with: 30 of 249
Skipping: 31 due to <20 min or not classic
Finished with: 32 of 249
Finis

Finished with: 242 of 249
Finished with: 243 of 249
Skipping: 244 due to <20 min or not classic
Finished with: 245 of 249
Skipping: 246 due to <20 min or not classic
Finished with: 247 of 249
Skipping: 248 due to <20 min or not classic
Finished with: 249 of 249
Finished with: 0 of 249
Skipping: 1 due to <20 min or not classic
Skipping: 2 due to <20 min or not classic
Finished with: 3 of 249
Finished with: 4 of 249
Finished with: 5 of 249
Finished with: 6 of 249
Finished with: 7 of 249
Skipping: 8 due to <20 min or not classic
Skipping: 9 due to <20 min or not classic
Finished with: 10 of 249
Finished with: 11 of 249
Skipping: 12 due to <20 min or not classic
Finished with: 13 of 249
Finished with: 14 of 249
Finished with: 15 of 249
Skipping: 16 due to <20 min or not classic
Skipping: 17 due to <20 min or not classic
Skipping: 18 due to <20 min or not classic
Finished with: 19 of 249
Skipping: 20 due to <20 min or not classic
Skipping: 21 due to <20 min or not classic
Skipping: 22 due t

Finished with: 234 of 249
Skipping: 235 due to <20 min or not classic
Finished with: 236 of 249
Skipping: 237 due to <20 min or not classic
Finished with: 238 of 249
Skipping: 239 due to <20 min or not classic
Skipping: 240 due to <20 min or not classic
Skipping: 241 due to <20 min or not classic
Finished with: 242 of 249
Finished with: 243 of 249
Finished with: 244 of 249
Skipping: 245 due to <20 min or not classic
Skipping: 246 due to <20 min or not classic
Skipping: 247 due to <20 min or not classic
Finished with: 248 of 249
Skipping: 249 due to <20 min or not classic
Finished with: 0 of 249
Finished with: 1 of 249
Skipping: 2 due to <20 min or not classic
Finished with: 3 of 249
Finished with: 4 of 249
Skipping: 5 due to <20 min or not classic
Skipping: 6 due to <20 min or not classic
Finished with: 7 of 249
Finished with: 8 of 249
Finished with: 9 of 249
Finished with: 10 of 249
Skipping: 11 due to <20 min or not classic
Skipping: 12 due to <20 min or not classic
Finished with: 13

Finished with: 217 of 249
Skipping: 218 due to <20 min or not classic
Skipping: 219 due to <20 min or not classic
Skipping: 220 due to <20 min or not classic
Skipping: 221 due to <20 min or not classic
Finished with: 222 of 249
Finished with: 223 of 249
Finished with: 224 of 249
Skipping: 225 due to <20 min or not classic
Skipping: 226 due to <20 min or not classic
Skipping: 227 due to <20 min or not classic
Skipping: 228 due to <20 min or not classic
Skipping: 229 due to <20 min or not classic
Skipping: 230 due to <20 min or not classic
Skipping: 231 due to <20 min or not classic
Skipping: 232 due to <20 min or not classic
Finished with: 233 of 249
Skipping: 234 due to <20 min or not classic
Skipping: 235 due to <20 min or not classic
Finished with: 236 of 249
Skipping: 237 due to <20 min or not classic
Skipping: 238 due to <20 min or not classic
Skipping: 239 due to <20 min or not classic
Finished with: 240 of 249
Skipping: 241 due to <20 min or not classic
Finished with: 242 of 249


Finished with: 207 of 249
Skipping: 208 due to <20 min or not classic
Skipping: 209 due to <20 min or not classic
Skipping: 210 due to <20 min or not classic
Skipping: 211 due to <20 min or not classic
Skipping: 212 due to <20 min or not classic
Finished with: 213 of 249
Skipping: 214 due to <20 min or not classic
Skipping: 215 due to <20 min or not classic
Skipping: 216 due to <20 min or not classic
Skipping: 217 due to <20 min or not classic
Skipping: 218 due to <20 min or not classic
Skipping: 219 due to <20 min or not classic
Skipping: 220 due to <20 min or not classic
Finished with: 221 of 249
Finished with: 222 of 249
Skipping: 223 due to <20 min or not classic
Skipping: 224 due to <20 min or not classic
Skipping: 225 due to <20 min or not classic
Skipping: 226 due to <20 min or not classic
Skipping: 227 due to <20 min or not classic
Skipping: 228 due to <20 min or not classic
Skipping: 229 due to <20 min or not classic
Finished with: 230 of 249
Finished with: 231 of 249
Finished

...

Finished with: 202 of 249
Skipping: 203 due to <20 min or not classic
Skipping: 204 due to <20 min or not classic
Finished with: 205 of 249
Skipping: 206 due to <20 min or not classic
Skipping: 207 due to <20 min or not classic
Skipping: 208 due to <20 min or not classic
Skipping: 209 due to <20 min or not classic
Skipping: 210 due to <20 min or not classic
Skipping: 211 due to <20 min or not classic
Skipping: 212 due to <20 min or not classic
Skipping: 213 due to <20 min or not classic
Finished with: 214 of 249
Finished with: 215 of 249
Finished with: 216 of 249
Skipping: 217 due to <20 min or not classic
Finished with: 218 of 249
Finished with: 219 of 249
Skipping: 220 due to <20 min or not classic
Finished with: 221 of 249
Finished with: 222 of 249
Skipping: 223 due to <20 min or not classic
Skipping: 224 due to <20 min or not classic
Skipping: 225 due to <20 min or not classic
Finished with: 226 of 249
Finished with: 227 of 249
Skipping: 228 due to <20 min or not classic
Skipping: 

Finished with: 193 of 249
Skipping: 194 due to <20 min or not classic
Finished with: 195 of 249
Finished with: 196 of 249
Finished with: 197 of 249
Finished with: 198 of 249
Finished with: 199 of 249
Skipping: 200 due to <20 min or not classic
Skipping: 201 due to <20 min or not classic
Skipping: 202 due to <20 min or not classic
Skipping: 203 due to <20 min or not classic
Skipping: 204 due to <20 min or not classic
Skipping: 205 due to <20 min or not classic
Finished with: 206 of 249
Finished with: 207 of 249
Finished with: 208 of 249
Finished with: 209 of 249
Skipping: 210 due to <20 min or not classic
Skipping: 211 due to <20 min or not classic
Skipping: 212 due to <20 min or not classic
Skipping: 213 due to <20 min or not classic
Skipping: 214 due to <20 min or not classic
Skipping: 215 due to <20 min or not classic
Skipping: 216 due to <20 min or not classic
Finished with: 217 of 249
Finished with: 218 of 249
Finished with: 219 of 249
Finished with: 220 of 249
Finished with: 221 o

Finished with: 195 of 249
Skipping: 196 due to <20 min or not classic
Skipping: 197 due to <20 min or not classic
Skipping: 198 due to <20 min or not classic
Skipping: 199 due to <20 min or not classic
Skipping: 200 due to <20 min or not classic
Skipping: 201 due to <20 min or not classic
Skipping: 202 due to <20 min or not classic
Skipping: 203 due to <20 min or not classic
Finished with: 204 of 249
Finished with: 205 of 249
Finished with: 206 of 249
Finished with: 207 of 249
Finished with: 208 of 249
Finished with: 209 of 249
Finished with: 210 of 249
Skipping: 211 due to <20 min or not classic
Finished with: 212 of 249
Skipping: 213 due to <20 min or not classic
Skipping: 214 due to <20 min or not classic
Skipping: 215 due to <20 min or not classic
Skipping: 216 due to <20 min or not classic
Skipping: 217 due to <20 min or not classic
Skipping: 218 due to <20 min or not classic
Skipping: 219 due to <20 min or not classic
Skipping: 220 due to <20 min or not classic
Skipping: 221 due 

Finished with: 171 of 249
Finished with: 172 of 249
Finished with: 173 of 249
Skipping: 174 due to <20 min or not classic
Finished with: 175 of 249
Skipping: 176 due to <20 min or not classic
Finished with: 177 of 249
Skipping: 178 due to <20 min or not classic
Skipping: 179 due to <20 min or not classic
Skipping: 180 due to <20 min or not classic
Skipping: 181 due to <20 min or not classic
Skipping: 182 due to <20 min or not classic
Finished with: 183 of 249
Finished with: 184 of 249
Finished with: 185 of 249
Finished with: 186 of 249
Skipping: 187 due to <20 min or not classic
Finished with: 188 of 249
Finished with: 189 of 249
Finished with: 190 of 249
Finished with: 191 of 249
Finished with: 192 of 249
Skipping: 193 due to <20 min or not classic
Skipping: 194 due to <20 min or not classic
Skipping: 195 due to <20 min or not classic
Skipping: 196 due to <20 min or not classic
Skipping: 197 due to <20 min or not classic
Skipping: 198 due to <20 min or not classic
Skipping: 199 due to

Finished with: 151 of 249
Finished with: 152 of 249
Skipping: 153 due to <20 min or not classic
Skipping: 154 due to <20 min or not classic
Skipping: 155 due to <20 min or not classic
Skipping: 156 due to <20 min or not classic
Skipping: 157 due to <20 min or not classic
Skipping: 158 due to <20 min or not classic
Skipping: 159 due to <20 min or not classic
Skipping: 160 due to <20 min or not classic
Skipping: 161 due to <20 min or not classic
Skipping: 162 due to <20 min or not classic
Skipping: 163 due to <20 min or not classic
Skipping: 164 due to <20 min or not classic
Skipping: 165 due to <20 min or not classic
Skipping: 166 due to <20 min or not classic
Skipping: 167 due to <20 min or not classic
Skipping: 168 due to <20 min or not classic
Skipping: 169 due to <20 min or not classic
Skipping: 170 due to <20 min or not classic
Skipping: 171 due to <20 min or not classic
Finished with: 172 of 249
Finished with: 173 of 249
Skipping: 174 due to <20 min or not classic
Finished with: 1

In [8]:
extracted_data

,killsplayer_1,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_6,killsplayer_7,killsplayer_8,killsplayer_9,deathsplayer_1,...,chemtechdragon_team100,hextechdragon_team100,killsplayer_10,riftherald_team100,baron_team200,killsplayer_0,riftherald_team300,waterdragon_team200,earthdragon_team100,waterdragon_team100
0,2.0,4.0,4.0,3.0,4.0,3.0,7.0,3.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,3.0,2.0,4.0,2.0,11.0,NaN,NaN,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,4.0,4.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,1.0,2.0,5.0,1.0,4.0,1.0,2.0,4.0,6.0,...,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN
4,1.0,1.0,1.0,6.0,2.0,2.0,NaN,9.0,5.0,3.0,...,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,2.0,5.0,5.0,2.0,2.0,1.0,4.0,NaN,2.0,2.0,...,3.0,1.0,2.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN
1486,4.0,3.0,NaN,5.0,NaN,7.0,3.0,NaN,4.0,1.0,...,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1487,1.0,1.0,1.0,8.0,NaN,4.0,4.0,3.0,3.0,5.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1488,NaN,4.0,3.0,4.0,1.0,5.0,2.0,1.0,4.0,2.0,...,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
